In [1]:
import neptune
import numpy as np
import plotly.graph_objects as go
import os

In [2]:
NEPTUNE_API_TOKEN = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI5YzY5NDlmYy1lNzNhLTQ4NjEtODY2Ny1kODM4ZGYyMWFkMmYifQ\=\="

In [5]:
# Fill ids, legend titles and save path below
# run_ids = ["LLMRANDOM-1014", "LLMRANDOM-1982"]
run_ids = ["LLMRANDOM-1647","LLMRANDOM-1636"]
legend_titles = ["Mixture of Tokens", "Mixture of Tokens + learnable temperature"]
assert len(run_ids) == len(legend_titles)

# download data from Neptune
project_name = "pmtest/llm-random"
runs = [neptune.init_run(project=project_name, with_id=run_id, mode="read-only",api_token=NEPTUNE_API_TOKEN) for run_id in run_ids]
losses = []
for run, title in zip(runs,legend_titles):
    losses.append((title,run["loss"].fetch_values()))
    

# function for saving in qmd format
def to_html(plot, path):
    html = plot.to_html(include_plotlyjs=False)
    with open(path, 'w') as f:
        f.write(html)
    with open(path, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write("```{=html}".rstrip('\r\n') + '\n' + content + '\n' + "```")
    os.rename(path, '_' + path)

https://app.neptune.ai/pmtest/llm-random/e/LLMRANDOM-1647
https://app.neptune.ai/pmtest/llm-random/e/LLMRANDOM-1636


In [8]:
plot = go.Figure()
x_axes = ["step","timestamp"]

for x_axis in x_axes:
    plot.data = []
    final_baseline_value = None
    horizontal_line_start = 0
    horizontal_line_end = 0
    max_step = -1
    for run, (title,loss), legend_title in zip(runs, losses, legend_titles):
        x_var = loss[x_axis].tolist()
        if x_axis == "timestamp":
            x_var = [x - x_var[0] for x in x_var]
            x_var = [x.total_seconds()/3600 for x in x_var]
            # this bullshit is only needed for my run, cause Athena is fucked
            subtract = 0
            subtracts = []
            for i, xv in enumerate(x_var):
                if i>0:
                    diff = xv-x_var[i-1]
                    if diff>0.5: #if the machine fucking stalled for more than half hour
                        subtract += diff
                subtracts.append(subtract)
            x_var = [xv-d for xv,d in zip(x_var,subtracts)]
            
        final_len = 250 if x_axis == "step" else int(len(x_var)*25/27.5)
        x_var = x_var[:final_len]
        values = loss['value'].tolist()[:final_len]
        # 
        # if title == "Baseline":
        #     final_baseline_value = values[-1]
        #     horizontal_line_end = x_var[-1]
        # if title == "Mixture of Tokens":
        #     index = np.argmin((np.array(values)-final_baseline_value)**2)
        #     horizontal_line_start = x_var[index]
        
        local_max_step = max(x_var)
        max_step = max(max_step, local_max_step)
            
                
        plot.add_scatter(
            x=x_var,
            y=values,
            name=legend_title,
        )
        

        
    
        plot.update_layout(
            title={'text': f'Training Loss vs {"Step" if x_axis=="step" else "Time"}'},
            # center the title
            title_x=0.5,
            template = "plotly"
            )
    # plot.add_shape(
    # # horizontal line
    # dict(
    #     type="line",
    #     x0=horizontal_line_start,
    #     x1=horizontal_line_end,
    #     y0=final_baseline_value,
    #     y1=final_baseline_value,
    #     line=dict(color="red", width=0.5, dash="dot")
    #     )
    # )
    # plot.add_annotation(
    # x=0.7*horizontal_line_start + 0.3* horizontal_line_end,
    # y=final_baseline_value,
    # ax=0.,
    # ay=50,
    # arrowcolor="red",
    # arrowhead=3,
    # arrowwidth=0.5,
    # 
    # text="24% of training steps" if x_axis == "step" else "33% of training time",
    #     showarrow=True,
    #     font=dict(
    #         size=11,
    #         color="red"
    #     ),
    # )

    
    # x and y axe ranges can be modified here
    plot = plot.update_xaxes(range=[0, 90_000])
    plot = plot.update_yaxes(range=[3.5, 5])
    
    plot.update_xaxes(title_text='training step' if x_axis == "step" else "time (hours)")
    plot.update_yaxes(title_text='loss')
        
    plot.show()
    to_html(plot, "step.qmd" if x_axis == "step" else "time.qmd")